# Barrier Option Pricing Examples

This notebook demonstrates how to use the `BarrierOptionsPricer` class to price various types of barrier options using Monte Carlo simulation. We will showcase examples for:
- A Down-and-Out Call option.
- An Up-and-In Put option.
- An example using the continuous monitoring approximation.
- An example using antithetic variates for variance reduction.

The pricer simulates asset price paths using Geometric Brownian Motion (GBM) and calculates the option price along with a confidence interval.


In [1]:
import numpy as np
import matplotlib.pyplot as plt # Though not used heavily in this specific notebook, good to have
from typing import Tuple, Union, Optional
import time

# For confidence interval calculation - ensure scipy is installed
# If not, the class has a fallback, but scipy is preferred for accuracy.
try:
    from scipy import stats
except ImportError:
    print("Warning: scipy.stats not found. Using a basic approximation for confidence interval critical values.")
    # Basic fallback if scipy is not available (as in the original script)
    class stats_fallback:
        class norm:
            @staticmethod
            def ppf(q):
                if abs(q - 0.975) < 1e-3: # For 95% CI
                    return 1.96
                elif abs(q - 0.995) < 1e-3: # For 99% CI
                    return 2.576
                else:
                    # A very rough approximation if needed for other CIs
                    # This is not a robust general inverse CDF
                    import math
                    return math.sqrt(2) * math.erfinv(2*q - 1) if q > 0 and q < 1 else 0

    stats = stats_fallback # Use the fallback if scipy.stats is not available


# Set a seed for reproducibility
np.random.seed(42)


## BarrierOptionsPricer Class and Helper Function

The following cell contains the full `BarrierOptionsPricer` class definition and the `print_detailed_results` helper function from the `barrier_options_pricer.py` file. We include them here to make the notebook self-contained and runnable.


In [6]:
class BarrierOptionsPricer:
    """
    Monte Carlo pricer for barrier options using Geometric Brownian Motion.
    
    Supports all four types of barrier options (Down-and-Out, Up-and-Out, 
    Down-and-In, Up-and-In) for both calls and puts.
    """
    
    def __init__(self):
        """Initialize the pricer."""
        self.valid_option_types = [
            'down_and_out_call', 'down_and_out_put',
            'up_and_out_call', 'up_and_out_put',
            'down_and_in_call', 'down_and_in_put',
            'up_and_in_call', 'up_and_in_put'
        ]
    
    def simulate_gbm_paths(self, S0: float, r: float, sigma: float, T: float, 
                          N_sim: int, N_steps: int) -> np.ndarray:
        """
        Simulate asset price paths using Geometric Brownian Motion.
        """
        dt = T / N_steps
        drift = (r - 0.5 * sigma**2) * dt
        diffusion = sigma * np.sqrt(dt)
        random_shocks = np.random.normal(0, 1, (N_sim, N_steps))
        
        paths = np.zeros((N_sim, N_steps + 1))
        paths[:, 0] = S0
        
        for i in range(N_steps):
            paths[:, i + 1] = paths[:, i] * np.exp(drift + diffusion * random_shocks[:, i])
        
        return paths
    
    def apply_continuity_correction(self, B: float, sigma: float, T: float, 
                                  N_steps: int, option_type: str) -> float:
        """
        Apply continuity correction for discrete monitoring to approximate continuous barrier.
        """
        dt = T / N_steps
        # Beta factor for continuity correction (Broadie, Glasserman, Kou, 1997)
        beta_correction_factor = 0.5826 
        correction_amount = beta_correction_factor * sigma * np.sqrt(dt)
        
        B_adj = B
        if 'down' in option_type: # For down options
            if 'out' in option_type: # Down-and-Out
                B_adj = B * np.exp(-correction_amount) # Shift barrier slightly lower
            else: # Down-and-In
                B_adj = B * np.exp(correction_amount)  # Shift barrier slightly higher
        elif 'up' in option_type: # For up options
            if 'out' in option_type: # Up-and-Out
                B_adj = B * np.exp(correction_amount)  # Shift barrier slightly higher
            else: # Up-and-In
                B_adj = B * np.exp(-correction_amount) # Shift barrier slightly lower
        return B_adj
    
    def calculate_barrier_payoff(self, path: np.ndarray, K: float, B: float, 
                               option_type: str, T: float, r_rate: float, 
                               monitoring_type: str = 'discrete', sigma_for_correction: Optional[float] = None) -> float:
        """
        Calculate the payoff for a single path given the barrier conditions.
        """
        if option_type not in self.valid_option_types:
            raise ValueError(f"Invalid option type. Must be one of {self.valid_option_types}")
        
        B_eff = B # Effective barrier
        if monitoring_type == 'continuous_approx':
            if sigma_for_correction is None:
                raise ValueError("sigma_for_correction must be provided for continuous_approx monitoring.")
            N_steps = len(path) - 1
            B_eff = self.apply_continuity_correction(B, sigma_for_correction, T, N_steps, option_type)

        knocked_out = False
        knocked_in = False
        
        if 'down_and_out' in option_type and path[0] <= B_eff:
            return 0.0
        if 'up_and_out' in option_type and path[0] >= B_eff:
            return 0.0
        
        for price_step in path: # Path includes S0
            if 'down' in option_type:
                if price_step <= B_eff:
                    if 'out' in option_type:
                        knocked_out = True; break
                    if 'in' in option_type:
                        knocked_in = True; break 
            elif 'up' in option_type:
                if price_step >= B_eff:
                    if 'out' in option_type:
                        knocked_out = True; break
                    if 'in' in option_type:
                        knocked_in = True; break 
        
        S_T = path[-1]
        intrinsic_payoff = max(S_T - K, 0) if 'call' in option_type else max(K - S_T, 0)
        
        payoff = 0.0
        if 'out' in option_type and not knocked_out:
            payoff = intrinsic_payoff
        elif 'in' in option_type and knocked_in:
            payoff = intrinsic_payoff
            
        return payoff

    def monte_carlo_pricer(self, S0: float, K: float, B: float, T: float,
                          r_rate: float, sigma: float, option_type: str, 
                          N_sim: int, N_steps: int,
                          monitoring_type: str = 'discrete',
                          confidence_level: float = 0.95,
                          antithetic: bool = False) -> Tuple[float, float, float, dict]:
        """
        Price barrier options using Monte Carlo simulation.
        """
        start_time = time.time()
        
        actual_N_sim_for_shocks = N_sim
        if antithetic:
            if N_sim % 2 != 0:
                # Ensure N_sim is even for antithetic pairs by generating for N_sim+1 if odd, then using N_sim pairs.
                # Or, more simply, generate N_sim//2 original shock sets and their antithetics.
                actual_N_sim_for_shocks = (N_sim + 1) // 2 if N_sim % 2 != 0 else N_sim // 2
            else:
                actual_N_sim_for_shocks = N_sim // 2
        
        # Generate random shocks for GBM
        dt = T / N_steps
        drift = (r_rate - 0.5 * sigma**2) * dt
        diffusion = sigma * np.sqrt(dt)
        # Generate shocks for the number of unique paths needed
        random_shocks_orig = np.random.normal(0, 1, (actual_N_sim_for_shocks, N_steps))

        all_payoffs = []

        # Simulate original paths and calculate payoffs
        paths_orig = np.zeros((actual_N_sim_for_shocks, N_steps + 1))
        paths_orig[:, 0] = S0
        for i in range(N_steps):
            paths_orig[:, i + 1] = paths_orig[:, i] * np.exp(drift + diffusion * random_shocks_orig[:, i])
        
        for j in range(actual_N_sim_for_shocks):
            payoff_orig = self.calculate_barrier_payoff(paths_orig[j,:], K, B, option_type, T, r_rate, monitoring_type, sigma)
            all_payoffs.append(payoff_orig)

        # Simulate antithetic paths and calculate payoffs if enabled
        if antithetic:
            random_shocks_anti = -random_shocks_orig # Negate all original shocks
            paths_anti = np.zeros((actual_N_sim_for_shocks, N_steps + 1))
            paths_anti[:, 0] = S0
            for i in range(N_steps):
                paths_anti[:, i + 1] = paths_anti[:, i] * np.exp(drift + diffusion * random_shocks_anti[:, i])
            
            for j in range(actual_N_sim_for_shocks):
                payoff_anti = self.calculate_barrier_payoff(paths_anti[j,:], K, B, option_type, T, r_rate, monitoring_type, sigma)
                all_payoffs.append(payoff_anti)
        
        all_payoffs = np.array(all_payoffs)
        
        mean_payoff = np.mean(all_payoffs)
        option_price = mean_payoff * np.exp(-r_rate * T)
        
        std_payoffs = np.std(all_payoffs, ddof=1) 
        n_total_effective_paths = len(all_payoffs)
        
        standard_error_payoff = std_payoffs / np.sqrt(n_total_effective_paths)
        
        alpha_ci = 1 - confidence_level
        critical_value = stats.norm.ppf(1 - alpha_ci/2)
        
        margin_of_error_price = critical_value * standard_error_payoff * np.exp(-r_rate * T)

        confidence_lower = option_price - margin_of_error_price
        confidence_upper = option_price + margin_of_error_price
        
        computation_time = time.time() - start_time
        mc_error_price_val = standard_error_payoff * np.exp(-r_rate * T)
        
        barrier_hit_count = 0
        # For barrier hit stats, we need all paths that were used in payoff calculation
        if antithetic:
            # If antithetic, we effectively used paths_orig and paths_anti
            combined_paths_for_stats = np.vstack((paths_orig, paths_anti))
        else:
            combined_paths_for_stats = paths_orig

        for path_stat in combined_paths_for_stats:
            B_eff_stat = B
            if monitoring_type == 'continuous_approx':
                 B_eff_stat = self.apply_continuity_correction(B, sigma, T, N_steps, option_type)

            hit_this_path = False
            if 'down' in option_type:
                if np.any(path_stat <= B_eff_stat): hit_this_path = True
            elif 'up' in option_type:
                if np.any(path_stat >= B_eff_stat): hit_this_path = True
            if hit_this_path:
                barrier_hit_count +=1
        
        barrier_hit_percentage = (barrier_hit_count / len(combined_paths_for_stats)) * 100 if len(combined_paths_for_stats) > 0 else 0

        statistics = {
            'mean_payoff': mean_payoff,
            'std_payoff': std_payoffs,
            'standard_error_payoff': standard_error_payoff,
            'mc_error_price': mc_error_price_val,
            'barrier_hit_percentage': barrier_hit_percentage,
            'computation_time': computation_time,
            'effective_simulations_used': n_total_effective_paths, # Total payoffs averaged
            'confidence_used': confidence_level,
            'convergence_ratio': mc_error_price_val / option_price if abs(option_price) > 1e-9 else float('inf')
        }
        
        return option_price, confidence_lower, confidence_upper, statistics

# Helper function to print results neatly
def print_detailed_results(price: float, conf_lower: float, conf_upper: float, 
                          stats: dict, option_params: dict):
    """Print detailed results of the barrier option pricing."""
    print("\n" + "="*60)
    print("BARRIER OPTION PRICING RESULTS")
    print("="*60)
    
    print(f"\nOption Type: {option_params['option_type'].replace('_', ' ').title()}")
    print(f"Initial Asset Price (S0): ${option_params['S0']:.2f}")
    print(f"Strike Price (K): ${option_params['K']:.2f}")
    print(f"Barrier Level (B): ${option_params['B']:.2f}")
    print(f"Time to Maturity (T): {option_params['T']:.4f} years")
    print(f"Risk-free Rate (r): {option_params['r_rate']:.2%}") 
    print(f"Volatility (σ): {option_params['sigma']:.2%}")
    
    print(f"\nSimulation Parameters:")
    print(f"Requested Simulations (N_sim): {option_params['N_sim']:,}")
    print(f"Effective Simulations Used in Payoff Avg: {stats['effective_simulations_used']:,}")
    print(f"Number of Time Steps (N_steps): {option_params['N_steps']:,}")
    print(f"Monitoring Type: {option_params.get('monitoring_type', 'discrete').replace('_',' ').title()}")
    print(f"Antithetic Variates: {'Yes' if option_params.get('antithetic', False) else 'No'}")
    
    print(f"\n" + "-"*30)
    print("PRICING RESULTS")
    print("-"*30)
    print(f"Estimated Option Price: ${price:.6f}")
    print(f"{stats['confidence_used']*100:.0f}% Confidence Interval: [${conf_lower:.6f}, ${conf_upper:.6f}]")
    print(f"Monte Carlo Error (SEM of Price): ±${stats['mc_error_price']:.6f}")
    # Corrected line: use 'price' instead of 'option_price'
    if abs(price) > 1e-9 : 
        print(f"MC Error as % of Price: { (stats['mc_error_price']/price)*100:.3f}%")

    print(f"\n" + "-"*30)
    print("ADDITIONAL STATISTICS")
    print("-"*30)
    print(f"Mean Payoff (undiscounted): ${stats['mean_payoff']:.6f}")
    print(f"Std Dev of Payoffs: ${stats['std_payoff']:.6f}")
    print(f"Standard Error of Mean Payoff: ±${stats['standard_error_payoff']:.6f}")
    print(f"Barrier Hit Percentage: {stats['barrier_hit_percentage']:.2f}%")
    print(f"Computation Time: {stats['computation_time']:.3f} seconds")



## Instantiate the Pricer

First, we create an instance of our `BarrierOptionsPricer`.


In [7]:
pricer = BarrierOptionsPricer()


## Example 1: Down-and-Out Call Option (Discrete Monitoring)

Let's price a Down-and-Out Call option.
- The option becomes worthless if the asset price drops to or below the barrier level.
- Payoff at maturity is $\max(S_T - K, 0)$ if the barrier was not hit.


In [8]:
# Parameters for Example 1
params_ex1 = {
    'S0': 100.0,
    'K': 105.0,
    'B': 90.0,  # Barrier below S0
    'T': 1.0,
    'r_rate': 0.05,
    'sigma': 0.20,
    'option_type': 'down_and_out_call',
    'N_sim': 100000,
    'N_steps': 252, # Daily steps for 1 year
    'monitoring_type': 'discrete',
    'antithetic': False
}

print("Pricing Example 1: Down-and-Out Call (Discrete Monitoring)")
price_ex1, ci_lower_ex1, ci_upper_ex1, stats_ex1 = pricer.monte_carlo_pricer(**params_ex1)
print_detailed_results(price_ex1, ci_lower_ex1, ci_upper_ex1, stats_ex1, params_ex1)


Pricing Example 1: Down-and-Out Call (Discrete Monitoring)

BARRIER OPTION PRICING RESULTS

Option Type: Down And Out Call
Initial Asset Price (S0): $100.00
Strike Price (K): $105.00
Barrier Level (B): $90.00
Time to Maturity (T): 1.0000 years
Risk-free Rate (r): 5.00%
Volatility (σ): 20.00%

Simulation Parameters:
Requested Simulations (N_sim): 100,000
Effective Simulations Used in Payoff Avg: 100,000
Number of Time Steps (N_steps): 252
Monitoring Type: Discrete
Antithetic Variates: No

------------------------------
PRICING RESULTS
------------------------------
Estimated Option Price: $6.910306
95% Confidence Interval: [$6.830551, $6.990062]
Monte Carlo Error (SEM of Price): ±$0.040692
MC Error as % of Price: 0.589%

------------------------------
ADDITIONAL STATISTICS
------------------------------
Mean Payoff (undiscounted): $7.264605
Std Dev of Payoffs: $13.527742
Standard Error of Mean Payoff: ±$0.042778
Barrier Hit Percentage: 52.74%
Computation Time: 2.692 seconds


## Example 2: Up-and-In Put Option (Discrete Monitoring)

Now, let's price an Up-and-In Put option.
- The option becomes active (a standard European put) only if the asset price rises to or above the barrier level.
- Payoff at maturity is $\max(K - S_T, 0)$ if the barrier was hit.


In [9]:
# Parameters for Example 2
params_ex2 = {
    'S0': 100.0,
    'K': 95.0,
    'B': 110.0, # Barrier above S0
    'T': 0.5,   # 6 months
    'r_rate': 0.03,
    'sigma': 0.25,
    'option_type': 'up_and_in_put',
    'N_sim': 100000,
    'N_steps': 126, # Daily steps for 6 months
    'monitoring_type': 'discrete',
    'antithetic': False
}

print("\nPricing Example 2: Up-and-In Put (Discrete Monitoring)")
price_ex2, ci_lower_ex2, ci_upper_ex2, stats_ex2 = pricer.monte_carlo_pricer(**params_ex2)
print_detailed_results(price_ex2, ci_lower_ex2, ci_upper_ex2, stats_ex2, params_ex2)



Pricing Example 2: Up-and-In Put (Discrete Monitoring)

BARRIER OPTION PRICING RESULTS

Option Type: Up And In Put
Initial Asset Price (S0): $100.00
Strike Price (K): $95.00
Barrier Level (B): $110.00
Time to Maturity (T): 0.5000 years
Risk-free Rate (r): 3.00%
Volatility (σ): 25.00%

Simulation Parameters:
Requested Simulations (N_sim): 100,000
Effective Simulations Used in Payoff Avg: 100,000
Number of Time Steps (N_steps): 126
Monitoring Type: Discrete
Antithetic Variates: No

------------------------------
PRICING RESULTS
------------------------------
Estimated Option Price: $0.481644
95% Confidence Interval: [$0.467479, $0.495809]
Monte Carlo Error (SEM of Price): ±$0.007227
MC Error as % of Price: 1.501%

------------------------------
ADDITIONAL STATISTICS
------------------------------
Mean Payoff (undiscounted): $0.488923
Std Dev of Payoffs: $2.319975
Standard Error of Mean Payoff: ±$0.007336
Barrier Hit Percentage: 55.28%
Computation Time: 1.795 seconds


## Example 3: Down-and-Out Call (Continuous Monitoring Approximation)

This example uses the same parameters as Example 1 but applies a continuity correction to approximate continuous barrier monitoring.


In [10]:
# Parameters for Example 3 (same as Ex1 but with continuous_approx)
params_ex3 = params_ex1.copy() # Start with params from example 1
params_ex3['monitoring_type'] = 'continuous_approx'
params_ex3['option_type'] = 'down_and_out_call' # Ensure it's set correctly

print("\nPricing Example 3: Down-and-Out Call (Continuous Monitoring Approximation)")
price_ex3, ci_lower_ex3, ci_upper_ex3, stats_ex3 = pricer.monte_carlo_pricer(**params_ex3)
print_detailed_results(price_ex3, ci_lower_ex3, ci_upper_ex3, stats_ex3, params_ex3)

print(f"\nNote: The effective barrier for continuous approximation was adjusted internally.")
print(f"Original Barrier: {params_ex3['B']:.4f}")
# For display, calculate the adjusted barrier again (the pricer does this internally)
adj_B_display = pricer.apply_continuity_correction(
    params_ex3['B'], params_ex3['sigma'], params_ex3['T'], 
    params_ex3['N_steps'], params_ex3['option_type']
)
print(f"Adjusted Barrier (approx): {adj_B_display:.4f}")



Pricing Example 3: Down-and-Out Call (Continuous Monitoring Approximation)

BARRIER OPTION PRICING RESULTS

Option Type: Down And Out Call
Initial Asset Price (S0): $100.00
Strike Price (K): $105.00
Barrier Level (B): $90.00
Time to Maturity (T): 1.0000 years
Risk-free Rate (r): 5.00%
Volatility (σ): 20.00%

Simulation Parameters:
Requested Simulations (N_sim): 100,000
Effective Simulations Used in Payoff Avg: 100,000
Number of Time Steps (N_steps): 252
Monitoring Type: Continuous Approx
Antithetic Variates: No

------------------------------
PRICING RESULTS
------------------------------
Estimated Option Price: $7.218128
95% Confidence Interval: [$7.137101, $7.299155]
Monte Carlo Error (SEM of Price): ±$0.041341
MC Error as % of Price: 0.573%

------------------------------
ADDITIONAL STATISTICS
------------------------------
Mean Payoff (undiscounted): $7.588209
Std Dev of Payoffs: $13.743394
Standard Error of Mean Payoff: ±$0.043460
Barrier Hit Percentage: 49.86%
Computation Time: 

## Example 4: Up-and-In Put (with Antithetic Variates)

This example uses similar parameters to Example 2 but enables antithetic variates for variance reduction.


In [11]:
# Parameters for Example 4 (same as Ex2 but with antithetic variates)
params_ex4 = params_ex2.copy() # Start with params from example 2
params_ex4['antithetic'] = True
params_ex4['N_sim'] = 100000 # Note: if N_sim is odd, pricer makes it even for antithetic

print("\nPricing Example 4: Up-and-In Put (with Antithetic Variates)")
price_ex4, ci_lower_ex4, ci_upper_ex4, stats_ex4 = pricer.monte_carlo_pricer(**params_ex4)
print_detailed_results(price_ex4, ci_lower_ex4, ci_upper_ex4, stats_ex4, params_ex4)

print(f"\nNote: Antithetic variates were used. Effective number of unique random shock sequences was {params_ex4['N_sim']//2 if params_ex4['N_sim']%2==0 else (params_ex4['N_sim']+1)//2}, but {stats_ex4['effective_simulations_used']} total payoffs were averaged.")



Pricing Example 4: Up-and-In Put (with Antithetic Variates)

BARRIER OPTION PRICING RESULTS

Option Type: Up And In Put
Initial Asset Price (S0): $100.00
Strike Price (K): $95.00
Barrier Level (B): $110.00
Time to Maturity (T): 0.5000 years
Risk-free Rate (r): 3.00%
Volatility (σ): 25.00%

Simulation Parameters:
Requested Simulations (N_sim): 100,000
Effective Simulations Used in Payoff Avg: 100,000
Number of Time Steps (N_steps): 126
Monitoring Type: Discrete
Antithetic Variates: Yes

------------------------------
PRICING RESULTS
------------------------------
Estimated Option Price: $0.487971
95% Confidence Interval: [$0.473518, $0.502424]
Monte Carlo Error (SEM of Price): ±$0.007374
MC Error as % of Price: 1.511%

------------------------------
ADDITIONAL STATISTICS
------------------------------
Mean Payoff (undiscounted): $0.495346
Std Dev of Payoffs: $2.367115
Standard Error of Mean Payoff: ±$0.007485
Barrier Hit Percentage: 55.43%
Computation Time: 1.485 seconds

Note: Antithe

## Conclusion

This notebook demonstrated the use of the `BarrierOptionsPricer` to estimate prices for different types of barrier options. We explored:
- Pricing of 'out' and 'in' options.
- The impact of discrete vs. approximated continuous barrier monitoring.
- The use of antithetic variates for potential variance reduction.

The Monte Carlo method provides a flexible framework for pricing such path-dependent options. The accuracy of the estimate can be improved by increasing the number of simulations and/or using variance reduction techniques. The confidence interval gives an indication of the precision of the estimated price.
